# Pendientes:
1. Asegurar que las tablas de resultados de qualifyers y carreras tengas las llaves necesarias para cruzarse. **Por piloto, id de carrera, temporada y/o fecha**
2. Crear un historial con 20 a 30 carreras anteriores, definir una base de datos para almacenarla y no reprocesar

# Consolidacion de datos de F1

La idea de este archivo es probar el flujo de las funciones creadas. Para traer los datos de la API, preprocesarlos y tener una tabla final consolidada con las distintas fuentes de datos.

In [3]:
# Module imports
from funcs_data_extraction import *
import pandas as pd
import numpy as np

## 1. Extraccion de datos de resultados de Qualifiers y de Carreras

**Definicion de parametros iniciales del modelo**

In [6]:
# Se calculara los resultados con las 10 carreras anteriores.
season_years = [2021, 2022, 2023]

### 1.1 Extraccion y transformacion de datos de Qualifiers

In [8]:
# Codigo para extraer la informacion de de qualifyers en las temporadas definidas inicialmente
# Se obtiene una lista de diccionarios
seasons = []
for year in season_years:
    seasons.append(get_season_qualifying_results(year))

In [9]:
# Se obtiene una lista de dataframes con los datos de cada ronda de las temporadas
quali_df_list = []
for s_idx, season_obj in enumerate(seasons):
    for i in range(len(season_obj)):
        quali_df_list.append(get_round_qualifying_results(season_obj, i+1))

In [10]:
quali_df_list = [df for df in quali_df_list if df is not None]
qualifyer_df = pd.concat(quali_df_list)

In [11]:
# Falta paso de preprocesar informacion para que quede en milisegundos los tiempos
def get_milisec_from_textTime(Q_time):
    if not Q_time:
        return None
    mins_s_and_ms = Q_time.split(':')
    mins = int(mins_s_and_ms[0])
    mins_in_ms = mins*60000
    ms_from_s_and_ms = float(mins_s_and_ms[1])*1000
    return int(mins_in_ms + ms_from_s_and_ms)


In [12]:
qualifyer_df['Q1_time_ms'] = qualifyer_df['best_time_Q1'].apply(get_milisec_from_textTime)
qualifyer_df['Q2_time_ms'] = qualifyer_df['best_time_Q2'].apply(get_milisec_from_textTime)
qualifyer_df['Q3_time_ms'] = qualifyer_df['best_time_Q3'].apply(get_milisec_from_textTime)

In [13]:
# extraccion de informacion
# Define the conversion dictionary
convert_dict = {'season': int, 'round_number': int}
qualifyer_df = qualifyer_df.astype(convert_dict)

In [14]:
# Función para agregar horas y convertir a Timedelta
def agregar_horas(t):
    if t is None:
        return pd.NaT
    try:
        # Separar minutos y segundos.milisegundos
        minutos, segundos_milisegundos = t.split(':')
        # Formatear el tiempo agregando horas "00:"
        nuevo_formato = f"00:{int(minutos):02}:{segundos_milisegundos}"
        return nuevo_formato
    except ValueError:
        return None
        
qualifyer_df['best_time_Q1_TD'] = qualifyer_df['best_time_Q1'].apply(agregar_horas)
qualifyer_df['best_time_Q1_TD'] = pd.to_timedelta(qualifyer_df['best_time_Q1_TD'])

qualifyer_df['best_time_Q2_TD'] = qualifyer_df['best_time_Q2'].apply(agregar_horas)
qualifyer_df['best_time_Q2_TD'] = pd.to_timedelta(qualifyer_df['best_time_Q2_TD'])

qualifyer_df['best_time_Q3_TD'] = qualifyer_df['best_time_Q3'].apply(agregar_horas)
qualifyer_df['best_time_Q3_TD'] = pd.to_timedelta(qualifyer_df['best_time_Q3_TD'])

In [15]:
# Melted con datos timedelta
# Transformacion de datos de los Qi_time_ms
timedeltas_melted_df = qualifyer_df.melt(id_vars=['season', 'race_name', 'fullname'],
                              value_vars=['best_time_Q1_TD', 'best_time_Q2_TD', 'best_time_Q3_TD'],
                              var_name='numero_quali', value_name='tiempo_TD'
                             )

# Ajustar la columna 'numero_quali' para que solo contenga el número de clasificación
timedeltas_melted_df['numero_quali'] = timedeltas_melted_df['numero_quali'].str.extract('(\d)').astype(int)

In [16]:
# Transformacion de datos de los Qi_time_ms
melted_df = qualifyer_df.melt(id_vars=['season', 'race_name', 'fullname'],
                              value_vars=['Q1_time_ms', 'Q2_time_ms', 'Q3_time_ms'],
                              var_name='numero_quali', value_name='tiempo_ms'
                             )

# Ajustar la columna 'numero_quali' para que solo contenga el número de clasificación
melted_df['numero_quali'] = melted_df['numero_quali'].str.extract('(\d)').astype(int)

In [17]:
# Merge de tablas melted
final_melted_df = melted_df.merge(timedeltas_melted_df, how='left', on=['season', 'race_name', 'fullname', 'numero_quali'])
# final_melted_df.head()

In [18]:
# Ahora encontramos el tiempo menor agrupando por season, race_name y numero_quali

mejor_en_qualis_df = final_melted_df.groupby(['season','race_name','numero_quali'])\
                            .agg(tiempo_min_ms = pd.NamedAgg(column='tiempo_ms', aggfunc='min')
                                 ,tiempo_min_TD = pd.NamedAgg(column='tiempo_TD', aggfunc='min')
                            )
mejor_en_qualis_df = mejor_en_qualis_df.sort_values(by='tiempo_min_ms', ascending=False)

In [19]:
# Cruce con tabla inicial
# esas mismas llaves nos permiten ponenr el valor al lado para luego calcular la resta de tiempo_ms - mejor_tiempo_ms
table_with_best_per_quali_df = final_melted_df.merge(mejor_en_qualis_df, how='left'
                                                  , on=["season", "race_name", "numero_quali"]
                                                 )
table_with_best_per_quali_df['diff_ms_with_1st_in_quali'] = table_with_best_per_quali_df['tiempo_ms'] - table_with_best_per_quali_df['tiempo_min_ms']
table_with_best_per_quali_df['diff_TD_with_1st_in_quali'] = table_with_best_per_quali_df['tiempo_TD'] - table_with_best_per_quali_df['tiempo_min_TD']

In [20]:
# Aca se busca donde la diferencia en milisegundos sea negativa con el primero para dejar en NaN el tiempo en milisegundos
table_with_best_per_quali_df.loc[table_with_best_per_quali_df['diff_ms_with_1st_in_quali']<0, 'diff_ms_with_1st_in_quali'] = np.nan
table_with_best_per_quali_df.loc[table_with_best_per_quali_df['diff_ms_with_1st_in_quali'].isna(), 'tiempo_ms'] = np.nan

# Como la columna en milisegundoso ya tiene NaN (los que no tienen resultados de quali y quedaron en 0 tras alguuna transformacion)
# Ahora buscamos esas filas del diff en ms donde hay nan y el timedelta lo dejamos en NaN 
table_with_best_per_quali_df.loc[table_with_best_per_quali_df['diff_ms_with_1st_in_quali'].isna(), 'diff_TD_with_1st_in_quali'] = np.nan
table_with_best_per_quali_df.loc[table_with_best_per_quali_df['diff_ms_with_1st_in_quali'].isna(), 'tiempo_TD'] = np.nan

In [21]:
# Función para convertir Timedelta a formato "HH:MM:SS.fff"
def convertir_a_formato(timedelta):
    if timedelta is np.nan or timedelta is pd.NaT:
        return None
    total_seconds = timedelta.total_seconds()
    hours = int(total_seconds // 3600)
    minutes = int((total_seconds % 3600) // 60)
    seconds = int(total_seconds % 60)
    milliseconds = int((total_seconds % 1) * 1000)
    return f"{hours:02}:{minutes:02}:{seconds:02}.{milliseconds:03}"

In [22]:
table_with_best_per_quali_df['tiempo_TD'] = table_with_best_per_quali_df['tiempo_TD'].apply(convertir_a_formato)
table_with_best_per_quali_df['diff_TD_with_1st_in_quali'] = table_with_best_per_quali_df['diff_TD_with_1st_in_quali'].apply(convertir_a_formato)

# Pendiente
**Principal**:
-  corregir diff_TD_with_1st_in_quali a formato de Power BI

**Segundario**
- Corregir procesos para que sea en mas sencillo la transformacion al formato adecuado de Power BI

In [44]:
# Exportacion ded la tabla agrupada de los tiempos
table_with_best_per_quali_df.to_csv('grouped_season_race_drive_nq.csv', index=False)